# Regularisation, Parameter Initialisation, Batchnorm, Optimisers

Create and compare different models (as described below).

Inspect the results by using tensorboard.


In [1]:
import os
import itertools
import datetime
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
x_train = x_train.reshape(-1, 28**2)
x_test = x_test.reshape(-1, 28**2)

### Parameters


In [4]:
layersizes = [50,50,50,10]
batchsize = 32 
epochs = 20
learning_rate = 0.1

tensorboard_folder = "tb_logs_keras"
outdir = os.path.join(os.getcwd(), tensorboard_folder)

### Baseline Model

* No regularisation
* No Batch Norm
* Default parameter initialisation of Keras: What is the default?
   - glorot_uniform
   - src:https://keras.io/api/layers/core_layers/dense/
* Sigmoid activation (last layer always softmax)
* SGD with given batchsize and learning rate, no accelerators (no momentum nor RMS prop).

Now, create the baseline model. 

Possibly, add convenient naming to the layers so that you can more easily read the outputs in tensorboard. 

In [5]:
def baseline_model(layersizes, activation):
    """
    Provides an MLP model (using Sequential) with given layersizes. The last layer is a softmax layer.
    As activation function use sigmoid.
        
    Arguments:
    layersizes -- list of integers with the number of hidden units per layer. The last element is for MNIST 10.
    activation -- string specifying the activation function for the hidden layers to be used.
    
    """
    ### START YOUR CODE HERE ###

    inputs = tf.keras.Input(shape=(28**2,), name="input_layer")
    x = inputs
    
    hidden_layer_nr = 1
    for size in layersizes[:-1]:
        x = tf.keras.layers.Dense(units=size, activation=activation, name=f"hidden_layer_{hidden_layer_nr}")(x)
        hidden_layer_nr += 1
    
    x = tf.keras.layers.Dense(units=layersizes[-1], activation="softmax", name="final_layer")(x)
    
    ### STOP YOUR CODE HERE ###

    return tf.keras.models.Model(inputs=inputs, outputs=x)

#### Run model

Use cross entropy as loss function.

In [6]:
run_name = "baseline"
rundir = os.path.join(outdir, run_name)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=rundir, histogram_freq=1, profile_batch=0)
# start tensorboard on command line with tensorboard -logs <path to outdir> 


### START YOUR CODE HERE ###

model = baseline_model(layersizes, "sigmoid")
model.summary()
model.compile(optimizer=tf.keras.optimizers.SGD(lr=learning_rate),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

history = model.fit(x_train, y_train,
                    batch_size=batchsize, epochs=epochs,
                    validation_split=0.2,
                    callbacks=[tensorboard_callback])


### STOP YOUR CODE HERE ###

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 784)]             0         
_________________________________________________________________
hidden_layer_1 (Dense)       (None, 50)                39250     
_________________________________________________________________
hidden_layer_2 (Dense)       (None, 50)                2550      
_________________________________________________________________
hidden_layer_3 (Dense)       (None, 50)                2550      
_________________________________________________________________
final_layer (Dense)          (None, 10)                510       
Total params: 44,860
Trainable params: 44,860
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1500/1500 [==============================] - 2s 1ms/step - loss: 2.2164 - accuracy: 0.1767 - val_loss: 1.0005 - 

<img src="img/acc1.png" style="float:left">
<img src="img/loss1.png">

We observe that the validation loss of the network is converging and the model already achieves an validation accuracy of  96.45%. With more epochs the model could gain another percent of accuracy.

### Parameter Initialisation

* No regularisation
* No Batch Norm
* __Parameter Initialisation: Compare GlorotNormal, Random Normal (mean 0, stdev 1), Zero, HeNormal__
* __Sigmoid Activation (last layer always softmax): Compare Sigmoid, ReLu__
* SGD with given batchsize and learning rate, no accelerators (no momentum nor RMS prop).

Hence, for each of the 4 initializers train and test a model sigmoid and relu.

In [7]:
def model_param_init(layersizes, initializer, activation):
    """
    Provides an MLP model (using Sequential) with given layersizes. The last layer is a softmax layer.
    As activation function use sigmoid.
        
    Arguments:
    layersizes -- list of integers with the number of hidden units per layer. The last element is for MNIST 10.
    initializer -- weight initializer
    activation -- string specifying the activation function to be used.
    
    """
    ### START YOUR CODE HERE ###

    inputs = tf.keras.Input(shape=(28**2,), name="input_layer")
    x = inputs
    
    hidden_layer_nr = 1
    for size in layersizes[:-1]:
        x = tf.keras.layers.Dense(units=size, activation=activation,
                                  name=f"hidden_layer_{hidden_layer_nr}",
                                  kernel_initializer=initializer,
                                  bias_initializer=initializer)(x)
        hidden_layer_nr += 1
    
    x = tf.keras.layers.Dense(units=layersizes[-1], activation="softmax", name="final_layer")(x)
    
    ### STOP YOUR CODE HERE ###

    return tf.keras.models.Model(inputs=inputs, outputs=x)

#### Run model

Run with the different settings.
Don't forget to configure the proper tensorboard callback.

In [8]:
activations = ["sigmoid", "relu"]

# initializers
glorot_norm = tf.keras.initializers.GlorotNormal()
glorot_norm.name = 'glorot_normal'

rand_norm = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05) # default params
rand_norm.name = 'random_normal'

rand_stand_norm = tf.keras.initializers.RandomNormal(mean=0.0, stddev=1.0) # default params
rand_stand_norm.name = 'random_stand_normal'

zero = tf.keras.initializers.Zeros()
zero.name = 'zero'

he_normal = tf.keras.initializers.HeNormal()
he_normal.name = 'he_normal'

initializers = [glorot_norm, rand_norm, rand_stand_norm, zero, he_normal]

for act, init in itertools.product(activations, initializers):
    print(40*"-")
    print(f"Activation {act}")
    print(40*"-")
    
    run_name = f"act-{act}-init-{init.name}"
    rundir = os.path.join(outdir, run_name)

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=rundir, histogram_freq=1, profile_batch=0)

    model = model_param_init(layersizes, init, act)
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=learning_rate),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    print(f"working on initializer '{init.name}'")
    history = model.fit(x_train, y_train,
                        batch_size=batchsize, epochs=epochs,
                        validation_split=0.2,
                        callbacks=[tensorboard_callback], verbose=False)

    print(f"latest val_loss: {history.history['val_loss'][-1]}")
    print(f"latest val_accuracy: {history.history['val_accuracy'][-1]}")
    print(40*"-")

----------------------------------------
Activation sigmoid
----------------------------------------
working on initializer 'glorot_normal'
latest val_loss: 0.11762388795614243
latest val_accuracy: 0.9652500152587891
----------------------------------------
----------------------------------------
Activation sigmoid
----------------------------------------
working on initializer 'random_normal'
latest val_loss: 0.16827039420604706
latest val_accuracy: 0.9516666531562805
----------------------------------------
----------------------------------------
Activation sigmoid
----------------------------------------
working on initializer 'random_stand_normal'
latest val_loss: 0.24702580273151398
latest val_accuracy: 0.9246666431427002
----------------------------------------
----------------------------------------
Activation sigmoid
----------------------------------------
working on initializer 'zero'
latest val_loss: 1.485146164894104
latest val_accuracy: 0.4117499887943268
--------------

##  Sigmoid
<img src="img/acc-sigmoid.png" style="float:left; width:400px">
<img src="img/loss-sigmoid.png" style="width:400px">
<img src="img/legend-sigmoid.png" style="width:700px">

We observe that for `glorot_normal`, `he_normal` and `random_normal` the model converges relatively fast. The later shows the slowest convergence speed, which could be due to the fact that the sigmoid activation function has two saturating regions and the `random_normal` initialization does not account for the number of inputs. Hence it is more probable that the logit value can get very large (or very small) and result in a vanishing gradient.

<br>
<div style="color:red">
    
Maybe the difference from `random_normal` to the other two could also come from the fact that it has a much smaller standard deviation as we learned today. Knowing that we could also argument that using `random_normal` initialization the initial neuron activations are more likely to lie within the linear region of the sigmoid function. This would imply that the optimization process would start with a weaker (almost linear) model, as a superposition of linear function is a linear function as well. Therefore the magnitude of the change that needs to be applied to the weights is larger and hence requires more time. This argumentation would also explain why the val_accuracy is almost 0 after the first epoch.

This argumentation would be supported if using a `random_normal` initialization with mean 0 and std 1 would converge faster. 
    
TBD: Results?
</div>

The slowest convergence speed is observed with the `zero` weight initializer. From the chain rule we know that the gradient of the activation of the previous layer are calculated as 

$$
\frac{\partial L}{\partial \mathbf{A}[l-1]}=\frac{\partial L}{\partial \mathbf{Z}^{[l]}} \cdot \frac{\partial \mathbf{Z}^{[l]}}{\partial \mathbf{A}^{[l-1]}}=\left(\mathbf{W}^{[1]}\right)^{T} \cdot \frac{\partial L}{\partial \mathbf{Z}^{[l]}}
$$

and will be backpropagated through multiplication. Hence the gradient will vanish already at the last layer in the first training iteration and will only be able to be propagated back properly, once the weights in the last layer get pushed away from zero. This obviously takes some time. This "pushing away from 0" can also be seen as the network trying to predict equal probability for all classes, as the extracted features at the last hidden layer are completely random ($\sigma(0)=0.5$).

Most probably the weights of all layers are non zero at around epoch 6 which then finally enabled the network to learn something meaningful.

## Relu
<img src="img/acc-relu.png" style="float:left">
<img src="img/loss-relu.png">
<img src="img/legend-relu.png">

We observe that for `glorot_normal`, `he_normal` and `random_normal` the model converges relatively fast. The later does not differ from the former two as it was observed with the sigmoid activation function. This could be due to the fact that the relu activation function has only a single saturating region.

The `zero` weight initializer does not work at all with the relu activation function. This is due to the fact that $\text{relu}(0)=0$. Hence 

$$
\frac{\partial L}{\partial \mathbf{W}^{[l]}}=\frac{\partial L}{\partial \mathbf{Z}[l]} \cdot \frac{\partial \mathbf{Z}^{[l]}}{\partial \mathbf{W}^{[l]}}=\frac{\partial L}{\partial \mathbf{Z}^{[l]}}\left(\mathbf{A}^{[l-1]}\right)^{T}
$$

will always be zero and so will the weights. The only thing that the model can learn is to predict equal probability for each class. Thus the maximal accuracy that is can reach is ~10%.

<img src="img/acc-relu-zero.png" style="float:left; width:400px">
<img src="img/loss-relu-zero.png" style='width:400px'>
<img src="img/legend-relu-zero.png" style='width:700px'>

The loss fluctuates very slightly because the gradients of the weights at the last layer are the only ones that will never be zero. Thus those weights will be adapted slightly in every iteration which then results in those small fluctuations.

In order to see more differences between `glorot_normal`, `he_normal` we could try to create a network with very "thin" layers  (small number of neurons), such that the constant factor of the two techniques would gain more weight as it is the only difference between the two.

$$
\begin{aligned}
\sigma_{\text{Glorot}}=&\sqrt{\frac{2}{n_{\text {inputs }}+n_{\text {outputs }}}} \\
\sigma_{\text{He}}=\sqrt{2} &\sqrt{\frac{2}{n_{\text {inpurs }}+n_{\text {outputs }}}}
\end{aligned}
$$

The expectation would be that `glorot_normal` works better for sigmoid and `he_normal` better for relu activations.

### Batch Normalisation

* No regularisation
* __Batch Norm__: with / without 
* __Parameter Initialisation: Random Normal (0,1), GlorotNormal__
* __Activation: Compare Sigmoid, ReLu__
* SGD with given batchsize and learning rate, no accelerators (no momentum nor RMS prop).

Run with/without batchnorm in combination with sigmoid or relu (with GlorotNormal).<br>
Run with/without batchnorm in combination with GlorotNormal or RandomNormal (with sigmoid).<br>
Hence run 8 different models.

In [9]:
def model_batchnorm(layersizes, initializer, activation):
    """
    Provides an MLP model (using Sequential) with given layersizes. The last layer is a softmax layer.
    As activation function use sigmoid.
        
    Arguments:
    layersizes -- list of integers with the number of hidden units per layer. The last element is for MNIST 10.
    initializer -- weight initializer
    activation -- string specifying the activation function to be used.
    """
    ### START YOUR CODE HERE ###

    inputs = tf.keras.Input(shape=(28**2,), name="input_layer")
    x = inputs
    
    hidden_layer_nr = 1
    for size in layersizes[:-1]:
        x = tf.keras.layers.Dense(units=size, activation=None,
                                  name=f"hidden_layer_{hidden_layer_nr}",
                                  kernel_initializer=initializer,
                                  bias_initializer=initializer)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(activation)(x)
        hidden_layer_nr += 1
    
    x = tf.keras.layers.Dense(units=layersizes[-1], activation="softmax", name="final_layer")(x)  
    
    
    ### STOP YOUR CODE HERE ###

    return tf.keras.models.Model(inputs=inputs, outputs=x)

#### Run model

Run the different variants.

In [10]:
### START YOUR CODE HERE ###

for act in ["sigmoid", "relu"]:
    print(40*"-")
    print(f"Activation {act}")
    print(40*"-")
    for batch_norm, init in itertools.product([True, False], ["glorot_normal", "random_normal"]):
        run_name = f"batchnorm-{batch_norm}-act-{act}-init-{init}"
        rundir = os.path.join(outdir, run_name)

        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=rundir, histogram_freq=1, profile_batch=0)

        if batch_norm:
            model = model_batchnorm(layersizes, init, act)
        else:
            model = model_param_init(layersizes, init, act)
        
        model.compile(optimizer=tf.keras.optimizers.SGD(lr=learning_rate),
                      loss="sparse_categorical_crossentropy",
                      metrics=["accuracy"])

        print(f"working on initializer {init} with batch_norm {batch_norm}")
        history = model.fit(x_train, y_train,
                            batch_size=batchsize, epochs=epochs,
                            validation_split=0.2,
                            callbacks=[tensorboard_callback], verbose=False)
        
        print(f"latest val_loss: {history.history['val_loss'][-1]}")
        print(f"latest val_accuracy: {history.history['val_accuracy'][-1]}")
        print(40*"-")

### STOP YOUR CODE HERE ###

----------------------------------------
Activation sigmoid
----------------------------------------
working on initializer glorot_normal with batch_norm True
latest val_loss: 0.08843450993299484
latest val_accuracy: 0.9731666445732117
----------------------------------------
working on initializer random_normal with batch_norm True
latest val_loss: 0.09289657324552536
latest val_accuracy: 0.971916675567627
----------------------------------------
working on initializer glorot_normal with batch_norm False
latest val_loss: 0.11580841988325119
latest val_accuracy: 0.9664166569709778
----------------------------------------
working on initializer random_normal with batch_norm False
latest val_loss: 0.14083606004714966
latest val_accuracy: 0.9628333449363708
----------------------------------------
----------------------------------------
Activation relu
----------------------------------------
working on initializer glorot_normal with batch_norm True
latest val_loss: 0.08475083112716675
l

## Sigmoid

<img src="img/acc-sigmoid-batchnorm.png" style="float:left">
<img src="img/loss-sigmoid-batchnorm.png">
<img src="img/legend-sigmoid-batchnorm.png">


We observe for both initializers that the model using batchnorm converges *much* faster and also to a solution with a higher accuracy.

## Relu

<img src="img/acc-relu-batchnorm.png" style="float:left">
<img src="img/loss-relu-batchnorm.png">
<img src="img/legend-relu-batchnorm.png">

We again observe for both initializers that the model using batchnorm converges *a little* faster and also to a solution with a higher accuracy. But in general when using the relu activation function the convergence is again much faster in contrast to the sigmoid activation function.

## Weights
Thanks to the tensorboard log, we can analyze the distribution of the kernel weights and compare them for different models
<img src="img/batchnorm-activation-hists.png">

The upper row contains the histogram of the kernel of the second hidden layer for different models **not** using batchnorm and the lower row contains the histrograms of the same layer of the same models but with batchnorm.

We can see that using batch norm results in much more consistent weights distribution over the different models (notice x-axis ranges). Another interesting observation is that when using a `standard_normal` initialization, the initial standard deviation of the weights is much smaller compared to the `glorot_normal` initialization.

We can also analyse a plot where the time is on the x-axis and the weights distribution on the y-axis.

<img src="img/batchnorm-activation-timedist.png">

Here we observe the weights distribution of the kernel of the third hidden layer without (left) and with (right) batchnorm. The main differences are the scale and rate of change (especially in the beginning) of the distribution. Again it is nicely visible that the batchnorm layers help to make the weights more consistent and thus enable a smoother /stable learning process.

We can also argument that using batchnorm layers helps to make the loss surface smoother. In the plot above we can see that the rate of change in the weights distribution is smoother using batchnorm which implies that the gradient is smoother which can only be the case if the loss surface is as well.

In order to make the effects of batch norm more clear we could make the model deeper and observe the weights histograms in the deep layers. There the effect should be better visible.

### Optimizers

* No regularisation
* No BatchNorm 
* Parameter Initialisation: GlorotNormal
* Activation: ReLu
* Optimizers: Compare 
    * SGD with given batchsize and learning rate, no accelerators (no momentum nor RMS prop)
    * RmsProp
    * Momentum

Create an according model and train it with the different optimizers.

In [11]:
### START YOUR CODE HERE ###
import itertools

learning_rate = 0.1

for opt in ["SGD", "RMS prop", "Momentum", 'Adam']:
    print(40*"-")
    print(f"Optimizer {opt}")
    print(40*"-")
    run_name = f"optimizer-{opt}"
    rundir = os.path.join(outdir, run_name)

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=rundir, histogram_freq=1, profile_batch=0)

    model = model_param_init(layersizes, "glorot_normal", "relu")

    if opt == "SGD":
        optimizer = tf.keras.optimizers.SGD(lr=learning_rate)
    elif opt == "RMS prop":
        optimizer = tf.keras.optimizers.RMSprop(lr=0.1*learning_rate)
    elif opt == "Momentum":
        optimizer = tf.keras.optimizers.SGD(lr=0.1*learning_rate, momentum=0.9, nesterov=True)
    elif opt == 'Adam':
        optimizer = tf.keras.optimizers.Adam(lr=0.001*learning_rate)

    model.compile(optimizer=optimizer,
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    print(f"working on optimizer {opt}")
    history = model.fit(x_train, y_train,
                        batch_size=batchsize, epochs=epochs,
                        validation_split=0.2,
                        callbacks=[tensorboard_callback], verbose=False)

    print(f"latest val_loss: {history.history['val_loss'][-1]}")
    print(f"latest val_accuracy: {history.history['val_accuracy'][-1]}")
    print(40*"-")

### STOP YOUR CODE HERE ###

----------------------------------------
Optimizer SGD
----------------------------------------
working on optimizer SGD
latest val_loss: 0.13547353446483612
latest val_accuracy: 0.9730833172798157
----------------------------------------
----------------------------------------
Optimizer RMS prop
----------------------------------------
working on optimizer RMS prop
latest val_loss: 2.116881847381592
latest val_accuracy: 0.48091667890548706
----------------------------------------
----------------------------------------
Optimizer Momentum
----------------------------------------
working on optimizer Momentum
latest val_loss: 0.13555559515953064
latest val_accuracy: 0.9712499976158142
----------------------------------------
----------------------------------------
Optimizer Adam
----------------------------------------
working on optimizer Adam
latest val_loss: 0.11547625064849854
latest val_accuracy: 0.9659166932106018
----------------------------------------


## Results

<img src="img/acc-opt.png" style="float:left">
<img src="img/loss-opt.png">
<img src="img/legend-opt.png">

`RMS prop` shows the fastes convergence and `SGD` with momentum the most stable and best end result. For those two to convergce in the first place, a much smaller global learning rate was required. The reason that `RMS prop` overfitts more could be due to the fact that it can get stuck faster in a local minima where as `SGD` with momentum can escape thanks to the momentum term. In theory the `Adam` optimizer should solve this discrepancy.

From my point of view the differences of the optimizers is already nicely shown in this example. But for the differences to become more extreme, we could increase the number of trainable parameters of the model (increase neurons in layers and / or increase network depth).